In [ ]:
# To make sure you are on track, I put together a baseline approach where I 
# apply parameter sharing and local connections to our dataset

In [ ]:
# Datasets grabbed from https://www.kaggle.com/borismarjanovic/price-volume-data-for-all-us-stocks-etfs
# Let's look at the problem of forecasting stocks

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.neural_network import MLPRegressor

In [ ]:
# Let's import one of the datasets from the link.
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
df = pd.read_csv('aadr.us.csv')

numeric_cols = ['Open', 'High', 'Low', 'Close', 'Volume']


In [ ]:
# Let's visualize some of our data.

import plotly.graph_objects as go
fig = go.Figure(data=[go.Candlestick(x=df.index[:100],
                open=df['Open'][:100],
                high=df['High'][:100],
                low=df['Low'][:100],
                close=df['Close'][:100])])
fig.update_layout(
    title= {
        'text': '',
      'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'},
      font=dict(
        family="Courier New, monospace",
        size=20,
        color="#7f7f7f"
    )
    )
fig.show()

In [ ]:
df['Date'] = df.index


In [ ]:
x = df[numeric_cols]

In [ ]:
n_date = x.shape[0]

In [ ]:
def calc_y(df):
    y_future = df[['High', 'Low']].mean(1).iloc[1:].values
    y_past = df['Close'].iloc[:-1].values
    y_diff = (y_future - y_past)/y_past
    return y_diff

In [ ]:
x_trn, x_val = x.iloc[:np.floor(n_date * .8).astype(int)+1], x.iloc[np.floor(n_date * .8).astype(int):]
y_trn, y_val = [calc_y(x_set) for x_set in (x_trn, x_val)]
mean_vals = x_trn[numeric_cols].mean().mean()
x_trn = x_trn / mean_vals
x_val = x_val / mean_vals
x_trn, x_val = x_trn[:-1], x_val[:-1]
x_trn, x_val = x_trn.values, x_val.values

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, LeakyReLU, Flatten, Dense
tfph = tf.compat.v1.placeholder
tf.compat.v1.disable_eager_execution()
xph = tfph(tf.float32, shape = (None, None, 5))
relu = LeakyReLU()
yph = tfph(tf.float32, shape = (None, None,))
hidden_size = 256
lstm = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
hidden = lstm(xph)
lstm2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
hidden = lstm2(hidden)
lstm3 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
hidden = lstm3(hidden)
lstm4 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
output = lstm4(hidden)

flattened = Flatten()(yph)
# yhat = Dense(10)(yph)
yhat = tf.reduce_mean(output, 2)
yhat_last = yhat[:,-1]
loss = tf.reduce_mean(tf.square(yph[:,3:] - yhat[:,3:]))
# loss =tf.reduce_mean((yhat - ytrn)**2)
opt = tf.compat.v1.train.AdamOptimizer(learning_rate=1e-6).minimize(loss)
sess = tf.compat.v1.Session()
sess.run(tf.compat.v1.global_variables_initializer())

In [ ]:
def get_all_samples(x, length = 6):
    x_max = x.shape[0] - length
    x_samples = np.stack([x[start:start + length] for start in range(x_max)], 0)
    return x_samples
    

In [ ]:
tf.compat.v1.reset_default_graph()
def get_samples(x, y, length = 10):
    x_min = 0
    x_max = x.shape[0] - length
    starts = np.random.randint(x_min, x_max, size = 8)
    x_stacked = np.stack([x[start:start + length] for start in starts], axis=0)
    y_stacked = np.stack([y[start:start + length] for start in starts], axis=0)
    return x_stacked, y_stacked
for i in range(1000):
    x_sample, y_sample = get_samples(x_trn, y_trn)
    _, l2 = sess.run([opt, loss], {xph:x_sample, yph:y_sample})
    if i % 100 == 0:
        x_all_samples = get_all_samples(x_trn)
        yhat_pred = sess.run(yhat_last, {xph:x_all_samples})
        corr = np.corrcoef(yhat_pred, y_trn[:-6])[1,0]
        print('corr',corr, 'trn loss', l2)
        
        
        x_sample, y_sample = get_samples(x_val, y_val)
        l2_val = sess.run(loss, {xph:x_sample, yph:y_sample})
        x_all_samples_val = get_all_samples(x_val)
        yhat_pred_val = sess.run(yhat_last, {xph:x_all_samples_val})
        corr_val = np.corrcoef(yhat_pred_val, y_val[:-6])[1,0]
        print('corr_val',corr_val, 'val loss', l2_val)